<span style="font-size: 18px;">Downoloading images</span>

In [1]:
import os
import datetime
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import requests
import urllib.parse
from tqdm import tqdm
from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions, CRS, MimeType, Geometry)

# Define your username and password
username = ""  # Replace with your actual username
password = ""  # Replace with your actual password

# Define global variables for token and expiry
keycloak_token = None
token_expiry = None

# Define a function to get a Keycloak token
def get_keycloak_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Response from the server was: {r.json()}"
        )
    response_json = r.json()
    global token_expiry
    # Token expires in seconds
    token_expiry = datetime.datetime.utcnow() + datetime.timedelta(seconds=response_json["expires_in"])
    return response_json["access_token"]

# Define a function to get a valid Keycloak token
def get_valid_keycloak_token() -> str:
    global keycloak_token, token_expiry
    if keycloak_token is None or datetime.datetime.utcnow() >= token_expiry:
        keycloak_token = get_keycloak_token(username, password)
    return keycloak_token

def download_image(date_str):
    keycloak_token = get_valid_keycloak_token()
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    start_date = (date - datetime.timedelta(days=1)).strftime("%Y-%m-%dT00:00:00.000Z")
    end_date = (date + datetime.timedelta(days=1)).strftime("%Y-%m-%dT00:00:00.000Z")

    aoi = "POLYGON ((-7.505894 40.007116, -7.457485 40.007116, -7.458344 39.965828, -7.503834 39.964645, -7.505894 40.007116))"
    encoded_aoi = urllib.parse.quote(aoi)

    request_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter=Collection/Name eq 'SENTINEL-2' and "
        f"OData.CSC.Intersects(area=geography'SRID=4326;{encoded_aoi}') and "
        f"ContentDate/Start gt {start_date} and ContentDate/Start lt {end_date}"
    )

    # Make request and handle response
    response = requests.get(request_url)
    response.raise_for_status()
    json_response = response.json()

    filtered_data = [
        item for item in json_response.get("value", [])
        if len(item.get("Name", "").split("_")) >= 0
        and item["Name"].split("_")[1] == "MSIL1C"
        and item["Name"].split("_")[5] == "T29TPE"
    ]

    if not filtered_data:
        print(f"No images found for date {date_str}")
        return

    min_date = min(item["OriginDate"] for item in filtered_data)
    filtered_data = [item for item in filtered_data if item["OriginDate"] == min_date]

    filtered_df = pd.DataFrame.from_dict(filtered_data)
    session = requests.Session()
    session.headers.update({"Authorization": f"Bearer {keycloak_token}"})

    for index, row in filtered_df.iterrows():
        download_path = fr"D:\AliFarrokhi\Santa_Agueda\lvl1sen2\{row['Name']}.zip"
        
        # Check if file already exists
        if os.path.exists(download_path):
            print(f"File {row['Name']}.zip already exists. Skipping download.")
            continue

        url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({row['Id']})/$value"
        response = session.get(url, stream=True, allow_redirects=False)

        while response.status_code in (301, 302, 303, 307):
            url = response.headers["Location"]
            response = session.get(url, stream=True, allow_redirects=False)

        total_size = int(response.headers.get("Content-Length", 0))
        chunk_size = 1024

        with open(download_path, "wb") as p:
            with tqdm(total=total_size, unit="B", unit_scale=True, desc=f"Downloading {row['Name']}") as progress_bar:
                for data in response.iter_content(chunk_size=chunk_size):
                    p.write(data)
                    progress_bar.update(len(data))

# Read the CSV file into a DataFrame
df = pd.read_csv(r"D:\AliFarrokhi\Santa_Agueda\Turbidity_nocloud.csv")

# Create a new DataFrame with unique values in the "SDate" column
new_df = pd.DataFrame({'SDate': df['Date']}).drop_duplicates()

# Download images for each unique date in new_df
for date in new_df['SDate']:
    try:
        download_image(str(date))
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            print("Token expired. Refreshing token...")
            # Obtain a new token and retry the download
            get_valid_keycloak_token()
            download_image(str(date))
        else:
            print(f"Error downloading image for date {str(date)}: {str(e)}")


No images found for date 2017-01-01
File S2A_MSIL1C_20170105T112442_N0204_R037_T29TPE_20170105T112737.SAFE.zip already exists. Skipping download.
No images found for date 2017-01-07
No images found for date 2017-01-11
No images found for date 2017-01-14
File S2A_MSIL1C_20170115T112411_N0204_R037_T29TPE_20170115T112412.SAFE.zip already exists. Skipping download.
No images found for date 2017-01-18
No images found for date 2017-01-19
No images found for date 2017-01-20
No images found for date 2017-01-21
No images found for date 2017-01-22
No images found for date 2017-01-23
No images found for date 2017-01-24
File S2A_MSIL1C_20170125T112331_N0204_R037_T29TPE_20170125T112333.SAFE.zip already exists. Skipping download.
No images found for date 2017-01-28
No images found for date 2017-02-07
No images found for date 2017-02-09
No images found for date 2017-02-19
No images found for date 2017-02-20
No images found for date 2017-02-21
No images found for date 2017-02-22
No images found for da

File S2A_MSIL1C_20180301T112111_N0206_R037_T29TPE_20180301T114356.SAFE.zip already exists. Skipping download.
No images found for date 2018-03-05


No images found for date 2018-03-24
No images found for date 2018-03-25


File S2B_MSIL1C_20180326T112109_N0206_R037_T29TPE_20180326T132716.SAFE.zip already exists. Skipping download.
No images found for date 2018-03-28
No images found for date 2018-03-29


No images found for date 2018-04-02
No images found for date 2018-04-03


...

File S2A_MSIL1C_20220129T112321_N0400_R037_T29TPE_20220129T152419.SAFE.zip already exists. Skipping download.
No images found for date 2022-01-31
No images found for date 2022-02-01
No images found for date 2022-02-02
No images found for date 2022-02-05
No images found for date 2022-02-06
No images found for date 2022-02-07


File S2A_MSIL1C_20220208T112231_N0400_R037_T29TPE_20220208T132948.SAFE.zip already exists. Skipping download.
No images found for date 2022-02-11
No images found for date 2022-02-15
No images found for date 2022-02-17


No images found for date 2022-02-20
No images found for date 2022-02-21
No images found for date 2022-02-22


No images found for date 2022-03-02
No images found for date 2022-03-04


File S2B_MSIL1C_20220305T112109_N0400_R037_T29TPE_20220305T123224.SAFE.zip already exists. Skipping download.
No images found for date 2022-03-07


<span style="font-size: 18px;">AC sen2cor</span>

In [3]:
import subprocess
import csv
from tqdm import tqdm

# Path to Sen2Cor executable
sen2cor_path = r"D:\AliFarrokhi\Sen2Cor\Sen2Cor-02.11.00-win64"

# Path to CSV file containing the full paths
csv_file_path = r"D:\AliFarrokhi\Santa_Agueda\lvl1sen2\path.csv"
# Open and read the CSV file
with open(csv_file_path, mode='r') as file:
    csv_reader = csv.reader(file)
    total_rows = sum(1 for _ in csv_reader)
    file.seek(0)
    
 
    for row in tqdm(csv_reader, total=total_rows, unit=" SAFE folder"):
        # Get the full path to the SAFE folder from the CSV
        full_path_to_safe_folder = row[0]
        
        # Build the command to run Sen2Cor
        sen2cor_command = f"cd /d {sen2cor_path} && L2A_Process.bat {full_path_to_safe_folder}"
        
        try:
            # Run the command
            subprocess.run(sen2cor_command, shell=True, check=True)
            # Wait for the process to complete
            subprocess.run("pause", shell=True)
        
        except subprocess.CalledProcessError as e:
            print(f"Error{full_path_to_safe_folder}: {e}")


100%|██████████████████████████████████████████████████████████████████| 251/251 [60:29:14<00:00, 867.55s/ SAFE folder]
